## Core Analysis Implementation

### 1.1 Data Preparation and EDA:

In [ ]:
import pymc as pm
import arviz as az
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load and prepare data
df = pd.read_csv('BrentOilPrices.csv', parse_dates=['Date'])
df = df.sort_values('Date')
df = df.set_index('Date')

# Focus on recent decade for clearer event association
start_date = '2012-01-01'
recent_df = df.loc[start_date:].copy()

# Plot raw price series
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Raw price series
axes[0, 0].plot(recent_df.index, recent_df['Price'])
axes[0, 0].set_title('Brent Oil Prices (2012-2022)')
axes[0, 0].set_ylabel('USD per Barrel')
axes[0, 0].grid(True)

# 2. Log returns
recent_df['Log_Return'] = np.log(recent_df['Price']) - np.log(recent_df['Price'].shift(1))
axes[0, 1].plot(recent_df.index, recent_df['Log_Return'])
axes[0, 1].set_title('Daily Log Returns')
axes[0, 1].set_ylabel('Log Return')
axes[0, 1].grid(True)

# 3. Histogram of returns
axes[1, 0].hist(recent_df['Log_Return'].dropna(), bins=50, edgecolor='black')
axes[1, 0].set_title('Distribution of Log Returns')
axes[1, 0].set_xlabel('Log Return')
axes[1, 0].set_ylabel('Frequency')

# 4. Rolling volatility (30-day)
axes[1, 1].plot(recent_df.index, recent_df['Log_Return'].rolling(30).std())
axes[1, 1].set_title('30-Day Rolling Volatility')
axes[1, 1].set_ylabel('Volatility')
axes[1, 1].grid(True)

plt.tight_layout()
plt.savefig('eda_visualizations.png', dpi=300, bbox_inches='tight')
plt.show()

# Prepare data for modeling
price_data = recent_df['Price'].values
n = len(price_data)

## Build Bayesian Change Point Model:

In [ ]:
with pm.Model() as change_point_model:
    
    # Prior for change point location (uniform over all days)
    tau = pm.DiscreteUniform("tau", lower=0, upper=n-1)
    
    # Priors for means before and after change point
    mu1 = pm.Normal("mu1", mu=np.mean(price_data), sigma=10)
    mu2 = pm.Normal("mu2", mu=np.mean(price_data), sigma=10)
    
    # Standard deviation (could also make this change)
    sigma = pm.HalfNormal("sigma", sigma=5)
    
    # Switch function: use mu1 before tau, mu2 after tau
    mean = pm.math.switch(tau > np.arange(n), mu1, mu2)
    
    # Likelihood - normal distribution
    likelihood = pm.Normal("likelihood", mu=mean, sigma=sigma, observed=price_data)
    
    # Sample from posterior
    trace = pm.sample(
        draws=2000,
        tune=1000,
        chains=4,
        cores=4,
        return_inferencedata=True,
        random_seed=42
    )

## Interpret Model Output:

In [ ]:
# Check convergence diagnostics
print("Convergence Diagnostics:")
print("=" * 50)
summary = az.summary(trace)
print(summary[['mean', 'sd', 'hdi_3%', 'hdi_97%', 'r_hat']])

# R-hat values close to 1.0 indicate good convergence
if all(summary['r_hat'] < 1.05):
    print("\n✓ All R-hat values < 1.05 - Good convergence achieved")
else:
    print("\n⚠ Some R-hat values > 1.05 - Consider more samples")

# Trace plots
az.plot_trace(trace)
plt.suptitle('Trace Plots for Model Parameters', y=1.02)
plt.tight_layout()
plt.savefig('trace_plots.png', dpi=300, bbox_inches='tight')
plt.show()

# Posterior distribution of tau
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# 1. Histogram of tau posterior
tau_samples = trace.posterior['tau'].values.flatten()
axes[0].hist(tau_samples, bins=50, edgecolor='black', alpha=0.7)
axes[0].axvline(np.mean(tau_samples), color='red', linestyle='--', label=f'Mean: {int(np.mean(tau_samples))}')
axes[0].set_xlabel('τ (index)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Posterior Distribution of Change Point (τ)')
axes[0].legend()
axes[0].grid(True)

# 2. Convert to actual dates
change_point_idx = int(np.mean(tau_samples))
change_point_date = recent_df.index[change_point_idx]
axes[1].plot(recent_df.index, recent_df['Price'], alpha=0.7)
axes[1].axvline(change_point_date, color='red', linestyle='--', 
                label=f'Change Point: {change_point_date.strftime("%Y-%m-%d")}')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Price (USD)')
axes[1].set_title('Price Series with Detected Change Point')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.savefig('change_point_detection.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\nDetected Change Point: {change_point_date.strftime('%Y-%m-%d')}")
print(f"Mean price before: ${trace.posterior['mu1'].mean().values:.2f}")
print(f"Mean price after: ${trace.posterior['mu2'].mean().values:.2f}")
print(f"Percentage change: {((trace.posterior['mu2'].mean().values - trace.posterior['mu1'].mean().values) / trace.posterior['mu1'].mean().values * 100):.1f}%")

## Associate Changes with Causes:

In [ ]:
# Load events data
events_df = pd.read_csv('historical_events.csv', parse_dates=['event_date'])

# Find closest event to detected change point
def find_closest_event(change_date, events_df, window_days=30):
    """Find events within a window of the change point"""
    closest_events = []
    for _, event in events_df.iterrows():
        days_diff = abs((change_date - event['event_date']).days)
        if days_diff <= window_days:
            closest_events.append({
                'event_name': event['event_name'],
                'event_date': event['event_date'].strftime('%Y-%m-%d'),
                'days_diff': days_diff,
                'event_type': event['event_type'],
                'expected_impact': event['expected_impact']
            })
    
    # Sort by proximity
    closest_events.sort(key=lambda x: x['days_diff'])
    return closest_events[:3]  # Return top 3 closest events

closest_events = find_closest_event(change_point_date, events_df)

print("\n" + "=" * 60)
print("EVENT ASSOCIATION ANALYSIS")
print("=" * 60)
print(f"Detected change point: {change_point_date.strftime('%Y-%m-%d')}")
print(f"\nClosest historical events (±30 days):")

for i, event in enumerate(closest_events, 1):
    print(f"\n{i}. {event['event_name']}")
    print(f"   Event Date: {event['event_date']}")
    print(f"   Days from change point: {event['days_diff']}")
    print(f"   Event Type: {event['event_type']}")
    print(f"   Expected Impact: {event['expected_impact']}")

# Quantify impact
mu1_mean = trace.posterior['mu1'].mean().values
mu2_mean = trace.posterior['mu2'].mean().values
price_change = mu2_mean - mu1_mean
percent_change = (price_change / mu1_mean) * 100

print(f"\n" + "=" * 60)
print("QUANTIFIED IMPACT")
print("=" * 60)
print(f"Mean price before change point: ${mu1_mean:.2f}")
print(f"Mean price after change point: ${mu2_mean:.2f}")
print(f"Absolute change: ${price_change:.2f}")
print(f"Percentage change: {percent_change:.1f}%")

# Generate impact statement
if closest_events:
    closest_event = closest_events[0]
    impact_statement = (
        f"Following {closest_event['event_name']} around {closest_event['event_date']}, "
        f"the model detects a structural break in Brent oil prices on {change_point_date.strftime('%Y-%m-%d')}. "
        f"The average daily price shifted from ${mu1_mean:.2f} to ${mu2_mean:.2f}, "
        f"representing a {percent_change:.1f}% {'increase' if percent_change > 0 else 'decrease'}."
    )
    print(f"\nIMPACT STATEMENT:\n{impact_statement}")

##  Incorporating Additional Factors:

In [ ]:
# Framework for multivariate analysis
def build_comprehensive_model(price_data, gdp_data=None, inflation_data=None):
    """Extended model with macroeconomic factors"""
    
    with pm.Model() as extended_model:
        
        # Multiple change points (extending to multiple breaks)
        n_changepoints = 3
        taus = []
        for i in range(n_changepoints):
            tau = pm.DiscreteUniform(f"tau_{i}", 
                                    lower=0 if i == 0 else taus[i-1]+1, 
                                    upper=len(price_data)-1)
            taus.append(tau)
        
        # Sort change points
        taus_sorted = pm.math.sort(pm.math.stack(taus))
        
        # Segment means
        mus = []
        for i in range(n_changepoints + 1):
            mu = pm.Normal(f"mu_{i}", mu=np.mean(price_data), sigma=20)
            mus.append(mu)
        
        # Create mean array based on change points
        mean_array = mus[0] * np.ones(len(price_data))
        for i, tau in enumerate(taus_sorted):
            mask = np.arange(len(price_data)) >= tau
            if i < len(mus) - 1:
                mean_array = pm.math.switch(mask, mus[i+1], mean_array)
        
        # Volatility clustering (GARCH-like component)
        sigma = pm.HalfNormal("sigma", sigma=5)
        
        # Likelihood
        likelihood = pm.Normal("likelihood", mu=mean_array, sigma=sigma, observed=price_data)
    
    return extended_model

##  Model Comparison Framework:

In [ ]:
def compare_models(models_dict, price_data):
    """Compare different change point models"""
    
    model_traces = {}
    model_metrics = {}
    
    for model_name, model in models_dict.items():
        print(f"\nFitting {model_name}...")
        
        with model:
            trace = pm.sample(draws=1000, tune=500, chains=2, random_seed=42)
            model_traces[model_name] = trace
            
            # Calculate WAIC for model comparison
            waic = az.waic(trace)
            model_metrics[model_name] = {
                'WAIC': waic.waic,
                'pWAIC': waic.p_waic,
                'WAIC_se': waic.waic_se
            }
    
    # Create comparison table
    comparison_df = pd.DataFrame(model_metrics).T
    print("\n" + "=" * 60)
    print("MODEL COMPARISON RESULTS")
    print("=" * 60)
    print(comparison_df)
    
    # Select best model (lowest WAIC)
    best_model = comparison_df['WAIC'].idxmin()
    print(f"\n✓ Best model: {best_model} (lowest WAIC)")
    
    return model_traces, comparison_df